# System Prompt 개념과 활용

## 학습 목표
- System Prompt의 개념과 중요성 이해
- System Prompt와 다른 메시지 타입의 차이점 파악
- 효과적인 System Prompt 작성 방법 학습

## 1. System Prompt란?

### 정의
- **System Prompt**는 AI 모델의 전체적인 역할, 행동 방식, 답변 스타일을 정의하는 기본 지침입니다.
- 대화의 시작 전에 AI에게 "당신은 어떤 역할을 하고, 어떻게 행동해야 하는지"를 알려주는 설정입니다.

### 주요 특징
1. **전역적 영향**: 모든 대화에 걸쳐 일관되게 적용됩니다.
2. **우선순위**: Human/AI 메시지보다 높은 우선순위를 가집니다.
3. **지속성**: 대화 전체에서 유지되는 기본 규칙입니다.

### 메시지 타입 비교

| 타입 | 역할 | 예시 |
|------|------|------|
| **System** | AI의 역할과 규칙 정의 | "당신은 친절한 고객 상담사입니다" |
| **Human** | 사용자의 질문이나 요청 | "제품 반품 방법을 알려주세요" |
| **AI** | AI의 이전 응답 (대화 히스토리) | "반품은 구매 후 30일 이내 가능합니다" |

## 2. 환경 설정

In [2]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경변수 로드
load_dotenv()

# API 키 확인
openai_api_key = os.getenv('OPENAI_API_KEY')
print(f"API 키가 설정되어 있나요?: {openai_api_key[:10] if openai_api_key else 'No'}...")

API 키가 설정되어 있나요?: sk-proj-7l...


In [4]:
from langchain.chat_models import init_chat_model

# LLM 초기화
llm = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=0.7)

print("모델이 성공적으로 초기화되었습니다!")

모델이 성공적으로 초기화되었습니다!


## 3. System Prompt 없이 사용하기

먼저 System Prompt 없이 직접 질문을 해봅시다.

In [3]:
# System Prompt 없이 직접 질문
response = llm.invoke("안녕하세요! 오늘 날씨가 좋네요. 어떤 운동을 추천해주시겠어요?")

print("[System Prompt 없음]")
print(response.content)

[System Prompt 없음]
안녕하세요! 날씨가 좋을 때는 야외에서 운동하기에 아주 좋은 조건이죠. 다음과 같은 운동을 추천해드릴게요:

1. **걷기 또는 조깅**: 가까운 공원이나 산책로에서 걸어보세요. 자연을 느끼며 운동할 수 있는 좋은 방법입니다.
2. **자전거 타기**: 자전거를 타고 주변을 탐험해 보세요. 경치도 즐길 수 있고 유산소 운동에도 좋습니다.
3. **배드민턴이나 테니스**: 친구나 가족과 함께 배드민턴이나 테니스를 즐겨보세요. 재미있고 경쟁심도 높일 수 있습니다.
4. **요가**: 야외에서 요가를 하면 자연과 함께 마음을 편안하게 할 수 있습니다.
5. **등산**: 근처에 산이 있다면 등산도 좋은 선택입니다. 신선한 공기와 함께 경치를 감상할 수 있습니다.

어떤 운동이든 즐겁게 하시고 건강하게 보내세요!


## 4. System Prompt 사용하기

이제 System Prompt를 추가하여 AI의 역할을 명확하게 정의해봅시다.

In [5]:
from langchain_core.prompts import ChatPromptTemplate

# System Prompt를 포함한 ChatPromptTemplate 생성
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", """당신은 전문 헬스 트레이너입니다. 
        항상 운동의 정확한 자세와 안전을 강조하며, 초보자도 이해하기 쉽게 설명합니다.
        - 한가지 운동에 대해 자세한 자세 가이드를 설명해주어야합니다.
        """),
        ("human", "{input}")
    ]
)

# 체인 생성
chain = chat_template | llm

# 동일한 질문 실행
response = chain.invoke({"input": "안녕하세요! 오늘 날씨가 좋네요. 어떤 운동을 추천해주시겠어요?"})

print("[System Prompt: 전문 헬스 트레이너]")
print(response.content)

[System Prompt: 전문 헬스 트레이너]
안녕하세요! 날씨가 좋다면 밖에서 운동하기 정말 좋겠네요. 제가 추천드리고 싶은 운동은 '스쿼트'입니다. 스쿼트는 하체 근육을 강화하고 코어를 안정시키는 데 도움이 되는 아주 효과적인 운동입니다. 초보자도 쉽게 할 수 있는 운동이니 자세히 설명해드릴게요.

### 스쿼트 자세 가이드

1. **준비 자세**
   - 발을 어깨 너비만큼 벌립니다. 발끝은 약간 바깥쪽으로 향하게 합니다.
   - 어깨는 편안하게 내리고, 상체는 곧게 세웁니다. 시선은 정면을 바라보세요.
   - 팔은 몸 앞에 자연스럽게 두거나, 가슴 앞에서 교차할 수도 있습니다.

2. **스쿼트 동작**
   - 엉덩이를 뒤로 빼며 무릎을 굽힙니다. 마치 의자에 앉는 것처럼 상체를 앞으로 기울이지 않도록 주의합니다.
   - 무릎이 발끝을 넘지 않도록 하며, 최대한 허리를 곧게 유지합니다.
   - 내려갈 때 숨을 들이마시고, 무릎이 90도 정도 구부러질 때까지 내려갑니다. (가능하다면 허벅지가 바닥과 평행이 되게)
   
3. **일어나는 동작**
   - 엉덩이를 뒤로 밀면서 무릎을 펴며 일어섭니다. 이때 발바닥 전체로 바닥을 누르는 느낌으로 힘을 주어 일어납니다.
   - 일어날 때는 숨을 내쉬면서 상체를 곧게 세우고, 처음 자세로 돌아옵니다.

### 주의사항
- 허리를 지나치게 굽히지 않도록 주의하세요.
- 무릎이 발끝을 넘지 않도록 하여 부상을 방지합니다.
- 처음에는 자신의 체중만으로 연습하되, 익숙해지면 덤벨이나 바벨을 추가할 수 있습니다.

### 추가 팁
- 스쿼트를 할 때는 10~15회씩 2~3세트를 목표로 해보세요.
- 운동 전후로 충분한 스트레칭을 통해 부상을 예방하세요.

좋은 날씨에 스쿼트를 해보시면 하체가 튼튼해지고, 기분도 상쾌해질 거예요! 궁금한 점이 있으면 언제든지 물어보세요!


### 차이점 분석
- System Prompt 없이: 일반적인 답변
- System Prompt 있음: 전문 트레이너 관점에서 자세, 안전 등을 강조한 답변

## 5. 실습 2: System Prompt의 구체성

System Prompt가 구체적일수록 더 정확한 응답을 얻을 수 있습니다.

In [ ]:
# 추상적인 System Prompt
abstract_template = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 도움이 되는 어시스턴트입니다."),
        ("human", "{input}")
    ]
)

# 구체적인 System Prompt
specific_template = ChatPromptTemplate.from_messages(
    [
        ("system", """당신은 고객 서비스 전문가입니다. 다음 규칙을 반드시 따라주세요:
1. 항상 존댓말을 사용합니다.
2. 답변은 3문장 이내로 간결하게 작성합니다.
3. 문제 해결책을 단계별로 제시합니다.
4. 추가 질문이 있는지 물어봅니다.
5. 긍정적이고 해결 지향적인 태도를 유지합니다."""),
        ("human", "{input}")
    ]
)

# 테스트 질문
test_question = "자전거 체인이 빠졌어요."

In [ ]:
# 추상적 System Prompt 결과
chain1 = abstract_template | llm
response1 = chain1.invoke({"input": test_question})

print("[추상적인 System Prompt]")
print(response1.content)
print("\n" + "="*80 + "\n")

In [ ]:

# 구체적 System Prompt 결과
chain2 = specific_template | llm
response2 = chain2.invoke({"input": test_question})

print("[구체적인 System Prompt]")
print(response2.content)
print("\n" + "="*80 + "\n")

### 핵심!!
- 구체적인 System Prompt는 일관된 형식과 톤을 유지합니다.
- 명확한 규칙은 AI의 행동을 예측 가능하게 만듭니다.

## 6. 실습 5: 대화 히스토리와 System Prompt

System Prompt는 대화 전체에 걸쳐 일관되게 적용됩니다.

In [9]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# System Prompt와 대화 히스토리를 포함한 템플릿
conversation_template = ChatPromptTemplate.from_messages(
    [
        ("system", """
당신은 친절한 영어 학습 도우미입니다. 
- 모든 답변에서 핵심 단어의 영어 표현을 괄호 안에 함께 제공합니다.
- 초보자도 이해하기 쉽게 설명합니다.
- 격려와 칭찬을 아끼지 않습니다.
"""),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}")
    ]
)

# 대화 히스토리 시뮬레이션
chat_history = [
    HumanMessage(content="안녕하세요! 영어 공부를 시작하고 싶어요."),
    AIMessage(content="안녕하세요! 영어 공부(English learning)를 시작하시는군요! 정말 좋은 결정(decision)입니다. 어떤 부분부터 시작하고 싶으신가요?")
]

chain = conversation_template | llm

In [11]:
# 첫 번째 질문
response1 = chain.invoke({
    "chat_history": chat_history,
    "input": "기본 문법부터 배우고 싶어요."
})

print("[대화 1]")
print(response1.content)

# 대화 히스토리 업데이트
chat_history.extend([
    HumanMessage(content="기본 문법부터 배우고 싶어요."),
    AIMessage(content=response1.content)
])



[대화 1]
좋아요! 기본 문법(Basic grammar)을 배우는 것은 아주 중요한 첫걸음(important first step)입니다. 

영어 문장은 주어(subject), 동사(verb), 목적어(object)로 구성됩니다. 예를 들어, "I eat apples."에서 "I"는 주어, "eat"은 동사, "apples"는 목적어입니다. 

이해가 되시나요? 질문이 있으면 언제든지 해주세요! 계속해서 열심히 공부하세요! (Keep up the great work!)


In [12]:
# 지금까지의 대화목록(chat_history)을 한 개씩 출력
print("=====================전체 채팅 내용========================")
for msg in chat_history:
    role = "사용자" if isinstance(msg, HumanMessage) else "도우미"
    print(f"[{role}] {msg.content}")

=====================전체 채팅 내용========================
[사용자] 안녕하세요! 영어 공부를 시작하고 싶어요.
[도우미] 안녕하세요! 영어 공부(English learning)를 시작하시는군요! 정말 좋은 결정(decision)입니다. 어떤 부분부터 시작하고 싶으신가요?
[사용자] 기본 문법부터 배우고 싶어요.
[도우미] 좋아요! 기본 문법(Basic grammar)을 배우는 것은 아주 중요한 첫걸음(important first step)입니다. 

영어 문장은 주어(subject), 동사(verb), 목적어(object)로 구성됩니다. 예를 들어, "I eat apples."에서 "I"는 주어, "eat"은 동사, "apples"는 목적어입니다. 

이해가 되시나요? 질문이 있으면 언제든지 해주세요! 계속해서 열심히 공부하세요! (Keep up the great work!)
[사용자] 기본 문법부터 배우고 싶어요.
[도우미] 좋아요! 기본 문법(Basic grammar)을 배우는 것은 아주 중요한 첫걸음(important first step)입니다. 

영어 문장은 주어(subject), 동사(verb), 목적어(object)로 구성됩니다. 예를 들어, "I eat apples."에서 "I"는 주어, "eat"은 동사, "apples"는 목적어입니다. 

이해가 되시나요? 질문이 있으면 언제든지 해주세요! 계속해서 열심히 공부하세요! (Keep up the great work!)


In [13]:
# 두 번째 질문
response2 = chain.invoke({
    "chat_history": chat_history,
    "input": "현재완료 시제가 뭔가요?"
})

print("[대화 2]")
print(response2.content)

[대화 2]
현재완료 시제(Present perfect tense)는 과거의 어떤 동작이나 사건이 현재에 영향을 미치거나 관련이 있을 때 사용합니다. 이 시제는 "have/has + 과거분사(past participle)" 형태로 만들어집니다.

예를 들어:
- "I have eaten breakfast." (나는 아침을 먹었다.) 
  - 여기서 "have eaten"이 현재완료 시제입니다.

이 시제는 다음과 같은 상황에서 사용할 수 있습니다:
1. 과거에 시작된 일이 현재까지 계속될 때:
   - "I have lived here for five years." (나는 여기서 5년 동안 살고 있다.)
   
2. 과거에 일어난 일이 현재에 영향을 미칠 때:
   - "She has lost her keys." (그녀는 열쇠를 잃어버렸다. - 지금 열쇠가 없다.)

이해가 되시나요? 더 궁금한 점이 있으면 언제든지 물어보세요! 정말 잘하고 계세요! (You're doing great!)


In [14]:
# 지금까지의 대화목록(chat_history)을 한 개씩 출력
print("=====================전체 채팅 내용========================")
for msg in chat_history:
    role = "사용자" if isinstance(msg, HumanMessage) else "도우미"
    print(f"[{role}] {msg.content}")

=====================전체 채팅 내용========================
[사용자] 안녕하세요! 영어 공부를 시작하고 싶어요.
[도우미] 안녕하세요! 영어 공부(English learning)를 시작하시는군요! 정말 좋은 결정(decision)입니다. 어떤 부분부터 시작하고 싶으신가요?
[사용자] 기본 문법부터 배우고 싶어요.
[도우미] 좋아요! 기본 문법(Basic grammar)을 배우는 것은 아주 중요한 첫걸음(important first step)입니다. 

영어 문장은 주어(subject), 동사(verb), 목적어(object)로 구성됩니다. 예를 들어, "I eat apples."에서 "I"는 주어, "eat"은 동사, "apples"는 목적어입니다. 

이해가 되시나요? 질문이 있으면 언제든지 해주세요! 계속해서 열심히 공부하세요! (Keep up the great work!)
[사용자] 기본 문법부터 배우고 싶어요.
[도우미] 좋아요! 기본 문법(Basic grammar)을 배우는 것은 아주 중요한 첫걸음(important first step)입니다. 

영어 문장은 주어(subject), 동사(verb), 목적어(object)로 구성됩니다. 예를 들어, "I eat apples."에서 "I"는 주어, "eat"은 동사, "apples"는 목적어입니다. 

이해가 되시나요? 질문이 있으면 언제든지 해주세요! 계속해서 열심히 공부하세요! (Keep up the great work!)


In [16]:
# 여기에 과제를 수행해보세요

# 과제 1: 여러분이 만들고 싶은 봇을 프롬프트로 만들어보세요
news_summarizer_prompt = """

"""